### 라이브러리 임포트

In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

### 전처리

In [3]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# 필셀 값을 0과 1 사이로 정규화
train_images, test_images = train_images / 255.0, test_images / 255.0

### 컨벌루션 신경망 생성
- keras에서는 6줄만 가지고 컨벌루션 신경망 생성 가능
- `Copy2D`와 `MaxPooling2D` 레이어 쌓기
- 입력: 이미지 높이, 이미지 너비, 채널 개수) 형식의 텐서
- 배치 개수는 항상 맨 앞에 있다고 가정
- MNIST 숫자 이미지 모양인 (28, 28, 1) 형상의 입력 처리하는 신경망 정의
- Conv2D와 MaxPooling 레이어의 출력: (높이, 너비, 채널 개수) 형식의 3차원 텐서
- 신경 모델의 마지막에는 항상 완전 연결 신경망인 Dense 레이어 추가 (3차원 -> 1차원)

In [4]:
model = models.Sequential() # 층을 순차적으로 쌓는 구조
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) # 첫번째 합성곱 층 추가. 필터 수 32개, 크기 3x3, 이미지 크기 28x28, 채널 수 1(흑백 이미지), 활성화함수 relu
model.add(layers.MaxPooling2D((2, 2))) # 2x2 크기의 최대 플링 층 추가하여 크기를 절반으로 축소
model.add(layers.Conv2D(64, (3, 3), activation='relu')) # 두 번째 합성곱 층. 필터 수 64개, 크기 3x3,  활성화함수 relu
model.add(layers.MaxPooling2D((2, 2))) # 두 번째 최대 풀링 층을 추가하여 크기로 절반으로 축소
model.add(layers.Conv2D(64, (3, 3), activation='relu')) # 세 번째 합성곱 층. 필터 수 64개, 크기 3x3, ReLU 활성화함수

model.add(layers.Flatten()) # 다차원 배열을 1차원으로 변환
model.add(layers.Dense(64, activation='relu')) # Dense 층 추가. 뉴런 수 64개, ReLU 활성화함수
model.add(layers.Dense(10, activation='softmax')) # 출력층: 10개의 클래스를 softmax로 분류

### 모델 출력
- Conv2D: 이미지 크기 2만큼 감소
- MaxPooling2D: 이미지의 크기 절반으로 감소

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

### 컴파일과 훈련
- 손실함수: sparse_categorical_crossentropy: 훈련 샘플의 출력을 원-핫 인코딩으로 만들지 않아도 됨

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5)


Epoch 1/5


1875/1875 [==============================] - 22s 11ms/step - loss: 0.1457 - accuracy: 0.9543
Epoch 2/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0473 - accuracy: 0.9853
Epoch 3/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0343 - accuracy: 0.9889
Epoch 4/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0255 - accuracy: 0.9921
Epoch 5/5
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0204 - accuracy: 0.9934


In [16]:
from sklearn.metrics import mean_squared_error, f1_score, recall_score, confusion_matrix, classification_report
import numpy as np

# 예측
pred = model.predict(test_images)
pred_labels = np.argmax(pred, axis=1)

# 손실과 정확도 출력
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"테스트 손실: {test_loss}")
print(f"테스트 정확도: {test_acc}\n")

print(confusion_matrix(test_labels, predicted_labels), "\n") # Confusion Matrix
print(classification_report(test_labels, pred_labels)) # 전체 지표

313/313 [==============================] - 3s 8ms/step - loss: 0.0302 - accuracy: 0.9911
테스트 손실: 0.03015991300344467
테스트 정확도: 0.991100013256073

[[ 978    0    0    0    0    0    0    1    1    0]
 [   0 1122    0    8    0    1    0    3    0    1]
 [   0    1 1029    0    0    0    0    2    0    0]
 [   0    0    0 1007    0    3    0    0    0    0]
 [   0    0    1    0  970    0    0    0    1   10]
 [   0    0    0   12    0  878    1    0    1    0]
 [   5    3    2    0    1    1  943    0    3    0]
 [   0    2    6    1    0    0    0 1018    0    1]
 [   0    0    3    1    0    0    0    1  968    1]
 [   1    0    1    3    1    3    0    0    2  998]] 

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       980
           1       0.99      0.99      0.99      1135
           2       0.99      1.00      0.99      1032
           3       0.98      1.00      0.99      1010
           4       1.00      0.99      0.99       9